In [12]:
import os
import glob
import numpy as np
from keras import optimizers
from keras.layers import LSTM
from keras.models import Sequential, Model
from keras.applications.vgg16 import VGG16
from keras.applications.mobilenet import MobileNet
from keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D

In [2]:
path = '../datasets/'
videoNames = os.listdir(path+'content/saliency/')
videoNames = [i[:-13] for i in videoNames]

In [3]:
# load the numpy arrays from saliency, motion maps and sensor data
sali = glob.glob(path+'content/saliencyImages/*.npy')
motion = glob.glob(path+'content/motionImages/*.npy')
sensory = glob.glob(path+'sensory/tile/*.npy')

In [4]:
videoNames = sorted(videoNames)[5:]
print(videoNames)
#print(sorted(sali))

for video in videoNames:
    npys = [s for s in sali if video in s]
    for npy in npys:
        data = np.load(npy)
        print(data.shape)
        break
    break

['landscape', 'pacman', 'panel', 'ride', 'sport']
(150, 1920, 3840)


In [16]:
# get the pre-trained VGG model
def loadVGG16Model():
    base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224,224,3))
    #print ("Model loaded..!")
    #print (base_model.summary())
    return base_model
vgg_model = loadVGG16Model()

In [13]:
def getBaseModel():
    #base_model  = MobileNet(input_shape=(128,128,3), include_top=False)
    base_model  = MobileNet(input_shape=(224,224,3), include_top=False)
    #base_model  = ResNet50(input_shape=(224,224,3), include_top=False)
    #base_model  = VGG16(input_shape=(224,224,3), include_top=False)
    #base_model  = InceptionV3(input_shape=(224,224,3), include_top=False)
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(1024, activation='relu')(x)
    predictions = Dense(2, activation='sigmoid')(x)
    model = Model(inputs=base_model.input, outputs=predictions)
    sgd = optimizers.SGD(lr=0.0001)
    model.compile(optimizer=sgd, loss='categorical_crossentropy')
    print (model.summary())
    return model
getBaseModel()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 112, 112, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 112, 112, 32)      128       
_________________________________________________________________
conv1_relu (Activation)      (None, 112, 112, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)      288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 112, 112, 32)      128       
_________________________________________________________________
conv_dw_1_relu (Activation)  (None, 112, 112, 32)      0         
__________